In [3]:
import numpy as np

In [4]:
def build_graph(fname):
    graph = dict()
    with open(fname) as infile:
        count = 0
        for line in infile:
            line = line.strip()
            if (count == 1):
                # nodes = line.split(',')
                nodes = line.split(';')
                for n in nodes:
                    graph.update({ n : [] })
            if (count > 3):
                if (line == ''):
                    break
                edge = line.split(' ')
                edge = edge[1:]
                # print(tuple(edge))
                if (edge[1] == '-->'):
                    graph[edge[0]].append(edge[2])
                elif (edge[1] == '<--'):
                    graph[edge[2]].append(edge[0])

            count += 1

    return graph

def ancestor(n1, n2, g):
    descendents = [n1]

    old = 0
    new = 1 
    while (old != new):
        for i in range(old, new):
            updates = g[descendents[i]]
            for n in g[descendents[i]]:
                if n in descendents:
                    updates.remove(n)
            descendents.extend(updates)

        # print(descendents)
        old = new
        new = len(descendents)

    return n2 in descendents

In [5]:
def load_edges(fname):
    rev_map = dict(
        {
            '-->' : '<--',
            '<--' : '-->',
            '<->' : '<->',
            'o->' : '<-o',
            '<-o' : 'o->',
            'o-o' : 'o-o',
            '---' : '---'
        }
    )

    print('loading edges from ' + fname)

    eef = open(fname, 'r')

    eeflines = eef.readlines()

    edges = list()

    linum = 0
    for line in eeflines:
        if (linum > 3):
            if line.strip()=='':
                break
            if line.strip()=='Ambiguous triples (i.e. list of triples for which there is ambiguous data about whether they are colliders or not):':
                break
            # print(line)
            temp = line.strip().split(' ')
            edge = [temp[1], temp[2], temp[3]]
            # edge.sort()
            # print(tuple(edge))
            if (edge[0] < edge[2]):
                edges.append(tuple(edge))
            else:
                edge.reverse()
                edge[1] = rev_map[edge[1]]
                edges.append(tuple(edge))
                
        linum+=1
        
    edges.sort()

    return edges

In [6]:
def calc_markov_equiv_arrow_pr(exp, true):
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)
    tn = np.zeros(7)
    
    for e1 in exp:
        # print(e1)
        for e2 in true:
            # print(e2)
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                idx = 0
                if ('X' in e1[0]) & ('X' in e1[2]):
                    idx = 1
                elif 'X' in e1[0]:
                    idx = 2
                elif 'Y' in e1[0]:
                    idx = 3
                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                    idx = 4
                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                    idx = 5
                for end in [0, 2]:
                    if e1[1][end] == '-':
                        if e2[1][end] != '-':
                            fn[0] += 1
                            fn[idx] += 1
                        else:
                            tn[0] += 1
                            tn[idx] += 1
                    elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                        if e2[1][end] == '-':
                            fp[0] += 1
                            fp[idx] += 1
                        else:
                            tp[0] += 1
                            tp[idx] += 1
            
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    tn[6] = tn[4] + tn[5]
    
#     print(tp)
#     print(fp)
#     print(fn)
#     print(tn)
    
    return tp/(tp+fp), tp/(tp+fn), (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [7]:
def calc_pag_arrow_pr(exp, pag, true): # true should be a graph from build_graph instead of a list of edges from load_edges so ancestry can be checked
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)

    for e1 in exp:
        for e2 in pag:
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                for end in [0, 2]:
                    if e2[1][end] == 'o':
                        if e1[1][end] == 'o':
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            if not ancestor(e1[end], e1[(end+2)%4], true):
                                tp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    tp[1] += 1
                                elif 'X' in e1[0]:
                                    tp[2] += 1
                                elif 'Y' in e1[0]:
                                    tp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    tp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    tp[5] += 1
                            else:
                                fn[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    fn[1] += 1
                                elif 'X' in e1[0]:
                                    fn[2] += 1
                                elif 'Y' in e1[0]:
                                    fn[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    fn[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    fn[5] += 1
                        elif (e1[1][end] == '-'):
                            if ancestor(e1[end], e1[(end+2)%4], true):
                                tp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    tp[1] += 1
                                elif 'X' in e1[0]:
                                    tp[2] += 1
                                elif 'Y' in e1[0]:
                                    tp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    tp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    tp[5] += 1
                            else:
                                fp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    fp[1] += 1
                                elif 'X' in e1[0]:
                                    fp[2] += 1
                                elif 'Y' in e1[0]:
                                    fp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    fp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    fp[5] += 1
                    elif (e2[1][end] == '>') | (e2[1][end] == '<'):
                        if (e1[1][end] == 'o'):
                            fp[0] += 0.5
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fp[1] += 0.5
                            elif 'X' in e1[0]:
                                fp[2] += 0.5
                            elif 'Y' in e1[0]:
                                fp[3] +=0.5
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fp[4] += 0.5
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fp[5] += 0.5
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '-'):
                            fp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fp[1] += 1
                            elif 'X' in e1[0]:
                                fp[2] += 1
                            elif 'Y' in e1[0]:
                                fp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fp[5] += 1
                    elif (e2[1][end] == '-'):
                        if (e1[1][end] == 'o'):
                            fn[0] += 0.5
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fn[1] += 0.5
                            elif 'X' in e1[0]:
                                fn[2] += 0.5
                            elif 'Y' in e1[0]:
                                fn[3] += 0.5
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fn[4] += 0.5
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fn[5] += 0.5
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '-'):
                            fn[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fn[1] += 1
                            elif 'X' in e1[0]:
                                fn[2] += 1
                            elif 'Y' in e1[0]:
                                fn[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fn[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fn[5] += 1
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    
    return tp/(tp+fp), tp/(tp+fn)

In [8]:
def calc_adj_pr(exp, true, numVars):
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)
    tn = 0.5 * numVars * (numVars-1) * np.ones(7)
    
    for e1 in true:
        idx = 0
        if ('X' in e1[0]) & ('X' in e1[2]):
            idx = 1
        elif 'X' in e1[0]:
            idx = 2
        elif 'Y' in e1[0]:
            idx = 3
        elif ('Survival' in e1[0]) & ('X' in e1[2]):
            idx = 4
        elif ('Survival' in e1[0]) & ('Y' in e1[2]):
            idx = 5
        fn[0] += 1
        fn[idx] +=1

    for e1 in exp:
        real = False
        for e2 in true:
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                real = True
                idx = 0
                if ('X' in e1[0]) & ('X' in e1[2]):
                    idx = 1
                elif 'X' in e1[0]:
                    idx = 2
                elif 'Y' in e1[0]:
                    idx = 3
                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                    idx = 4
                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                    idx = 5
                tp[0] += 1
                tp[idx] +=1
                break
        if (not real):
            idx = 0
            if ('X' in e1[0]) & ('X' in e1[2]):
                idx = 1
            elif 'X' in e1[0]:
                idx = 2
            elif 'Y' in e1[0]:
                idx = 3
            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                idx = 4
            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                idx = 5
            fp[0] += 1
            fp[idx] +=1
                
    fn -= tp
    tn -= (tp + fp + fn)
            
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    tn[6] = tn[4] + tn[5]
    
#     print(tp)
#     print(fp)
#     print(fn)
#     print(tn)
    
    return tp/(tp+fp), tp/(tp+fn), (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [15]:
true = build_graph('../fci-max/rCausalMGM/graph/graph.n1000.p100.txt')
pag = load_edges('../fci-max/rCausalMGM/graph/pag.n1000.p100.txt')
fcim = load_edges('../fci-max/cfci_test.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../fci-max/rCausalMGM/graph/pag.n1000.p100.txt
loading edges from ../fci-max/cfci_test.txt


In [16]:
prec, rec, mcc = calc_adj_pr(fcim, pag, 100)
print('adj precision:\t' + str(np.round(prec[0],3)))
print('adj recall:\t' + str(np.round(rec[0],3)))
print('adj MCC:\t' + str(np.round(mcc[0],3)))

adj precision:	0.979
adj recall:	0.92
adj MCC:	0.947


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [17]:
prec, rec = calc_pag_arrow_pr(fcim, pag, true)
print('arrow precision:  ' + str(np.round(prec[0],3)))
print('arrow recall:     ' + str(np.round(rec[0],3)))
# print('adj MCC:\t' + str(np.round(mcc[0],3)))

arrow precision:  0.904
arrow recall:     0.718


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in true_divide


In [18]:
# true = build_graph('../fci-max/rCausalMGM/graph/graph.n1000.p100.txt')
true = load_edges('../fci-max/rCausalMGM/graph/mec.n100.p25.txt')
pcm = load_edges('../fci-max/pcmax_test.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../fci-max/rCausalMGM/graph/mec.n100.p25.txt
loading edges from ../fci-max/pcmax_test.txt


In [19]:
prec, rec, mcc = calc_adj_pr(pcm, true, 25)
print('adj precision:\t' + str(np.round(prec[0],3)))
print('adj recall:\t' + str(np.round(rec[0],3)))
print('adj MCC:\t' + str(np.round(mcc[0],3)))

adj precision:	0.926
adj recall:	0.5
adj MCC:	0.641


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [20]:
prec, rec, mcc = calc_markov_equiv_arrow_pr(pcm, true)
print('arrow precision:  ' + str(np.round(prec[0],3)))
print('arrow recall:     ' + str(np.round(rec[0],3)))
print('adj MCC:\t' + str(np.round(mcc[0],3)))

arrow precision:  0.769
arrow recall:     0.4
adj MCC:	0.319


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide
